### Tokenization

Tokenization is the process of breaking down a sentence into smaller units or tokens, and index mapping is the mapping of these tokens onto an index. The GPT-2 (Generative Pre-trained Transformer) cache refers to the stored activations of the neural network layers. The cache for a specific layer in the GPT-2 model has a matrix of lower triangular tokens. Setting up the attention layer for this model involves defining the attention, layer, and return values. The layer_type_alias and act_name_alias are used to represent layer types and names, respectively. Understanding the GPT-2 cache is important for interpreting neural networks. Specifically, for each head in a particular layer, there is a matrix of lower triangular tokens that are used to set up attention. This layer then returns a specific type of pattern, such as attention or an MLP (multilayer perceptron).

In layer 0 attention patterns, it is possible to visualize patterns using the CircuitViz library. For example, the summarization token has the "ization" closely linked to "summar," and "supervised" is linked to a couple of tokens before it. Investigating the reason for these links is crucial.

### Hooks and Ablation

To interpret neural networks, it is possible to mess around with the internals and observe how the behavior changes. Inside transformers, there are hook points around each activation, and we can change or edit at the hooks. The model computes activation normally, and then applies the hook and computes again, substituting the original. Comparing with the PyTorch native hook function, hook code is janky, but when running the training process, the hook takes in a module (probably redundant), an input, and an output, and then calculates the mean and deviation on each activation on the current model. For registering a hook to a module, we register a forward hook function to a module.

Ablation involves deleting an activation function to see how it affects behavior and recording this change. Zero ablation or pruning is setting the output to zero, and in standard ML, pruning means deleting a weight rather than an activation. The issue with pruning is that the output might break. Mean ablation is visualized by having the activations as points on a circle, and the mean is then the center. Random ablation takes another data point and replaces the activation of the current data point with that. Models like GPT-2 are trained using dropout, so ablation might not work because it is, in a sense, "protected" by this pruning. Checking random ablation on object identification can help fix this issue.

### Defining Hook Functions and Activation Patching

Defining hook functions involves taking in a value and a hook point. A value has details of batch dimension, position, head_index, and d_head (model dim/size of the tensor). The utils.get_act_name("v", layer_to_ablate) function takes a string layer and returns an attribute that contains it for the forward hook.

Activation patching involves checking a corrupted prompt and a normal prompt to see what happens. It works best when token lengths are similar. For example comparing across, "The Eiffel Tower is in Paris" versus "The Colosseum is in.". The neuron also activates after feeding an "Apple or axe or…" prompt to it, probably meaning that there is an attention head that looks out for vowel patterns and is able to activate that neuron. For the an-neuron, we found an attention head that was significant in activation patching. It primarily looked at the "Apple" token, so our guess was that that attention head was probably picking up that information and activating the an-neuron downstream at the last token pos. There are probably attention head-neuron pairs out in the wild that make a neuron generalize to many more situations (e.g., vowel predictor neuron activating outside of "an" as the last word).

To simplify the interpretation process, the following type and activation name aliases are often used:


In [ ]:
layer_type_alias = {
        "a": "attn",
        "m": "mlp",
        "b": "",
        "block": "",
        "blocks": "",
        "attention": "attn",
    }

act_name_alias = {
        "attn":"pattern",
        "attn_logits":"attn_scores",
        "key":"k",
        "query":"q",
        "value":"v",
        "mlp_pre":"pre",
        "mlp_mid":"mid",
        "mlp_post":"post",
    }

Using loss as a metric for activation patching instead of logits can also be helpful. 